In [1]:
import numpy as np
from datetime import datetime
from strat import MACDandEMA

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    since_datetime=datetime(2024, 2, 19),
    until_datetime=datetime(2024, 2, 28),
    # candles_to_dl=3000,
)

In [3]:
candles.shape

(2592, 6)

In [4]:
macd_strat = MACDandEMA(
    long_short="long",
    ema_length=np.arange(200, 401, 200),
    fast_length=np.arange(10, 21, 10),
    macd_below=np.array([0]),
    signal_smoothing=np.arange(5, 16, 10),
    slow_length=np.arange(30, 61, 30),
)

In [5]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)



In [6]:
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [7]:
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([5]),
    sl_based_on_add_pct=np.array([0.1, 0.25]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [8]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 16
Total order settings to test: 16
Total combinations of settings to test: 256
Total candles: 2,592
Total candles to test: 663,552


In [9]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
1,0,1,8.0,2,6,10.160,25.000,0.364,179.880,1101.605,101.605
2,0,2,8.0,2,6,5.971,25.000,0.269,179.171,1059.707,59.707
3,0,3,8.0,2,6,5.971,25.000,0.269,179.171,1059.707,59.707
129,8,1,11.0,3,8,12.765,27.273,0.248,212.575,1127.648,127.648
0,0,0,8.0,2,6,2.747,25.000,0.194,178.537,1027.475,27.475
131,8,3,11.0,3,8,5.609,27.273,0.175,204.887,1056.085,56.085
130,8,2,11.0,3,8,5.609,27.273,0.175,204.887,1056.085,56.085
128,8,0,11.0,3,8,5.427,27.273,0.127,210.549,1054.269,54.269
9,0,9,6.0,1,5,-16.437,16.667,-0.697,80.000,835.627,-164.373
10,0,10,6.0,1,5,-19.012,16.667,-0.803,79.985,809.884,-190.116


In [10]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
    dos_index=15,
    ind_set_index=13,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 5.0,
    sl_based_on_add_pct = 0.0025,
    sl_based_on_lookback = 50,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.02,
)


In [11]:
order_records_df[order_records_df["order_status"] == "StopLossFilled"]

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
2,13,15,458,1708438200000,2024-02-20 14:10:00,StopLossFilled,1033.977,1033.977,NaN,NaN,...,NaN,NaN,52316.8,NaN,NaN,33.977,NaN,NaN,NaN,NaN
4,13,15,658,1708498200000,2024-02-21 06:50:00,StopLossFilled,1003.063,1003.063,NaN,NaN,...,NaN,NaN,51711.7,NaN,NaN,-30.914,NaN,NaN,NaN,NaN
6,13,15,1022,1708607400000,2024-02-22 13:10:00,StopLossFilled,973.216,973.216,NaN,NaN,...,NaN,NaN,51426.3,NaN,NaN,-29.847,NaN,NaN,NaN,NaN
11,13,15,1926,1708878600000,2024-02-25 16:30:00,StopLossFilled,857.236,857.236,NaN,NaN,...,NaN,NaN,51430.0,NaN,NaN,-115.980,NaN,NaN,NaN,NaN
